# Импортируем библиотеки

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.metrics import f1_score, r2_score
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.utils import resample
import matplotlib.pyplot as plt

# Вспомогательные функции

In [2]:
from sklearn.metrics import recall_score, precision_score, f1_score
def metrics_classification(model, y_test, pred, average=None):

    # Подсчет метрик качества модели
    recall = recall_score(y_test, pred, average=average).mean()
    precision = precision_score(y_test, pred, average=average).mean()
    f1 = f1_score(y_test, pred, average=average).mean()

    # Сохранение метрик в словарь
    metrics_dict = {
        'Model': model,
        'recall': recall,
        'precision': precision,
        'f1_score': f1
    }

    return metrics_dict

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd

def split_and_normalize_data(X, y, test_size=0.2, normalization='none'):

    # Разбиение данных на обучающий и тестовый наборы
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

    # Нормализация данных
    if normalization == 'standard':
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
    elif normalization == 'minmax':
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test


In [4]:
def find_optimal_k_for_knn(X, y, k_range, test_size=0.2, normalization='none', average=None):

    # Разбиение данных на обучающий и тестовый наборы
    X_train, X_test, y_train, y_test = split_and_normalize_data(X, y, test_size, normalization)

    # Инициализация списка для сохранения значений MSE
    metrics = {}
    matrix = []
    y_test = y_test.to_numpy()
    # Проход по каждому значению K
    for p in range(1, 3):
      for k in range(1, k_range):
          # Создание модели KNN с текущим значением K
          knn = KNeighborsClassifier(n_neighbors=k, p=p)

          # Обучение модели на обучающем наборе данных
          knn.fit(X_train, y_train)

          # Предсказание на тестовом наборе данных
          pred = knn.predict(X_test)

          metrics[k] = [metrics_classification(knn, y_test, pred, average=average)['f1_score'], k, p]

    # Находим оптимальное значение K
      best_k = max(metrics.values())

    return best_k

# Данные

## Импортируем данные

In [5]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ML/lab6/dataset-tortuga.csv')
data.shape

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/ML/lab6/dataset-tortuga.csv'

In [ ]:
data.head()

Unnamed: 0 - Бесполезный столбец

NAME - Имя ученика

USER_ID - ID ученика

HOURS_DATASCIENCE - Количество часов, потраченных на изучение курсов по DataScience

HOURS_BACKEND - Количество часов, потраченных на изучение курсов по Backend

HOURS_FRONTEND - Количество часов, потраченных на изучение курсов по Frontend

NUM_COURSES_BEGINNER_DATASCIENCE - Количество пройденных начальных курсов по DataScience

NUM_COURSES_BEGINNER_BACKEND - Количество пройденных начальных курсов по Backend

NUM_COURSES_BEGINNER_FRONTEND - Количество пройденных начальных курсов по Frontend

NUM_COURSES_ADVANCED_DATASCIENCE - Количество пройденных курсов для продвинутых по DataScience

NUM_COURSES_ADVANCED_BACKEND - Количество пройденных курсов для продвинутых по Backend

NUM_COURSES_ADVANCED_FRONTEND - Количество пройденных курсов для продвинутых по Frontend

AVG_SCORE_DATASCIENCE - Средний балл по DataScience, набранный учеником

AVG_SCORE_BACKEND - Средний балл по Backend, набранный учеником

AVG_SCORE_FRONTEND - Средний балл по Frontend, набранный учеником

PROFILE - Технический профиль студента (кто он по итогу)

## Предобработка

In [6]:
data.drop(columns=['USER_ID', 'Unnamed: 0', 'NAME'], axis=1, inplace=True)

NameError: name 'data' is not defined

In [ ]:
data.head()

In [7]:
data.info()

NameError: name 'data' is not defined

In [ ]:
data = data.fillna(0)

In [ ]:
data.info()

In [8]:
data['PROFILE'] = data['PROFILE'].factorize()[0]

NameError: name 'data' is not defined

In [ ]:
data.corr()

## Выбираем данные для модели

In [9]:
X = data.drop(columns=['PROFILE'])
y = data['PROFILE']

NameError: name 'data' is not defined

In [ ]:
y.unique()

In [ ]:
y.value_counts()

In [ ]:
y.value_counts().plot(kind='bar')

# Строим модель

## Обычный KNN

In [10]:
X_train, X_test, y_train, y_test = split_and_normalize_data(X, y, test_size=0.2, normalization='standard')

NameError: name 'X' is not defined

In [ ]:
best_k = find_optimal_k_for_knn(X, y, k_range=15, test_size=0.2, normalization='standard', average=None)
best_k

In [11]:
knn = KNeighborsClassifier(n_neighbors=12, p=2)
knn.fit(X_train, y_train)

NameError: name 'X_train' is not defined

In [ ]:
pred = knn.predict(X_test)

In [ ]:
metrics = metrics_classification(knn, y_test, pred, average=None)
metrics

In [ ]:
print(classification_report(y_test, pred))

## Взвешенный KNN

In [12]:
X_train, X_test, y_train, y_test = split_and_normalize_data(X, y, test_size=0.2, normalization='standard')

NameError: name 'X' is not defined

In [ ]:
best_k = find_optimal_k_for_knn(X, y, k_range=15, test_size=0.2, normalization='standard', average='weighted')
best_k

In [13]:
knn = KNeighborsClassifier(n_neighbors=12, p=2)
knn.fit(X_train, y_train)

NameError: name 'X_train' is not defined

In [14]:
pred = knn.predict(X_test)

NameError: name 'X_test' is not defined

In [ ]:
metrics = metrics_classification(knn, y_test, pred, average='weighted')
metrics

In [ ]:
print(classification_report(y_test, pred))

# Сохранение модели

In [15]:
import pickle

In [16]:
with open('/content/drive/MyDrive/Colab Notebooks/ML/lab6/model.pkl', 'wb') as f:
  pickle.dump(knn, f)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/ML/lab6/model.pkl'

# Открытие модели

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/ML/lab6/model.pkl', 'rb') as f:
  knn2 = pickle.load(f)

In [ ]:
pred = knn2.predict(X_test)

In [ ]:
print('predicted:', pred[:5])
print('true:', y_test[:5])

predicted: [4 4 1 0 0]
true: 10650    4
2041     4
8668     2
1114     0
13902    0
Name: PROFILE, dtype: int64


In [ ]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.93      0.96      0.94       654
           1       0.94      0.94      0.94       681
           2       0.93      0.95      0.94       651
           3       0.92      0.93      0.92       655
           4       0.93      0.91      0.92       710
           5       0.95      0.91      0.93       649

    accuracy                           0.93      4000
   macro avg       0.93      0.93      0.93      4000
weighted avg       0.93      0.93      0.93      4000

